**MILESTONE:1** Historical Stock Analysis, Visualisation and Insights


**Data Extraction**


In [1]:
import pandas as pd

In [2]:
stock_history_df = pd.read_csv("/content/stock_history.csv")
order_history_df = pd.read_csv("/content/order_history.csv")
safety_stock_data=pd.read_csv("/content/safety_stock_levels.csv")
warehouse_information=pd.read_csv("/content/warehouse_information.csv")

**Understock** **Detection**


Calculation the number of understock situations in different warehouses by comparing the stock levels of the products with their safety stock levels.

In [4]:
safety_stock_levels = safety_stock_data.set_index('product_id')['Safety_stock_level']
safety_stock_data = safety_stock_data.drop_duplicates(subset='product_id')
Understock = stock_history_df[stock_history_df['stock_level'] < stock_history_df['product_id'].map(safety_stock_levels)]
understock_situations = Understock.groupby('warehouse_id').size().reset_index(name='understock_count')
understock_situations['warehouse_id'] = pd.Categorical(understock_situations['warehouse_id'], categories=[f'W{i}' for i in range(1, 21)])
understock_situations_sorted = understock_situations.sort_values(by='warehouse_id')
print(understock_situations_sorted.to_string(index=False))

warehouse_id  understock_count
          W1                57
          W2                58
          W3                47
          W4                49
          W5                48
          W6                51
          W7                37
          W8                50
          W9                51
         W10                71
         W11                55
         W12                37
         W13                48
         W14                54
         W15                68
         W16                46
         W17                69
         W18                42
         W19                61
         W20                53


**Overstock Detection**


Calculation of the total stock quantity by date and warehouse, and identification of overstock situations where stock exceeds capacity.

In [5]:
import pandas as pd
stock_levels_df=pd.read_csv("/content/stock_history.csv")
warehouse_capacities_df=pd.read_csv("/content/warehouse_information.csv")
stock_levels_df['date'] = pd.to_datetime(stock_levels_df['date'])
grouped = stock_levels_df.groupby(['date', 'warehouse_id'])
total_quantity = grouped['stock_level'].sum()
merged_df = pd.merge(warehouse_capacities_df,total_quantity , on='warehouse_id')
overstock_df = merged_df[merged_df['stock_level'] > merged_df['capacity']]
overstock_count_by_warehouse = overstock_df.groupby('warehouse_id').size().reset_index(name='overstock_counts')
all_warehouse_ids = ['W{}'.format(i) for i in range(1, 21)]
all_warehouses_df = pd.DataFrame({'warehouse_id': all_warehouse_ids})
overstock_countsdf = pd.merge(all_warehouses_df, overstock_count_by_warehouse, on='warehouse_id', how='left')
overstock_countsdf['overstock_counts'].fillna(0, inplace=True)
print(overstock_countsdf[['warehouse_id', 'overstock_counts']].to_string(index=False))

warehouse_id  overstock_counts
          W1               0.0
          W2               0.0
          W3               0.0
          W4               0.0
          W5               0.0
          W6               0.0
          W7               0.0
          W8               0.0
          W9               0.0
         W10               0.0
         W11               0.0
         W12               0.0
         W13               0.0
         W14               0.0
         W15               0.0
         W16               0.0
         W17               0.0
         W18               1.0
         W19               0.0
         W20               0.0


**Out of Stock Detection**

Analysis of historical stock and order data to determine the count of times products were out of stock in each warehouse.

In [6]:
stock_history_df['date'] = pd.to_datetime(stock_history_df['date'])
order_history_df['date'] = pd.to_datetime(order_history_df['date'])
stock_history_df.sort_values(by=['date', 'warehouse_id', 'product_id'], inplace=True)
stock_history_df = stock_history_df.drop_duplicates(subset=['date', 'warehouse_id', 'product_id'], keep='last')
merged_df = pd.merge(order_history_df, stock_history_df, on=['date', 'warehouse_id', 'product_id'], how='left')
merged_df['is_out_of_stock'] = merged_df['quantity_ordered'] > merged_df['stock_level']
out_of_stock_df = merged_df[merged_df['is_out_of_stock']]
warehouse_out_of_stock_counts = out_of_stock_df.groupby('warehouse_id').size().reset_index(name='out_of_stock_count')
all_warehouse_ids = ['W{}'.format(i) for i in range(1, 21)]
all_warehouses_df = pd.DataFrame({'warehouse_id': all_warehouse_ids})
out_of_stock_counts = pd.merge(all_warehouses_df, warehouse_out_of_stock_counts, on='warehouse_id', how='left')
out_of_stock_counts['out_of_stock_count'].fillna(0, inplace=True)
print(out_of_stock_counts[['warehouse_id', 'out_of_stock_count']].to_string(index=False))

warehouse_id  out_of_stock_count
          W1                 2.0
          W2                 0.0
          W3                 3.0
          W4                 0.0
          W5                 1.0
          W6                 1.0
          W7                 0.0
          W8                 0.0
          W9                 0.0
         W10                 1.0
         W11                 1.0
         W12                 0.0
         W13                 1.0
         W14                 0.0
         W15                 1.0
         W16                 0.0
         W17                 1.0
         W18                 0.0
         W19                 1.0
         W20                 0.0


**Flagged** **Situations**

This code combines counts of overstock, understock, and out-of-stock situations for different warehouses, calculating the total flagged situations for each warehouse.

In [7]:
merged_counts = pd.merge(overstock_countsdf, understock_situations_sorted, on='warehouse_id', how='outer')
merged_counts = pd.merge(merged_counts, out_of_stock_counts, on='warehouse_id', how='outer')
merged_counts.fillna(0, inplace=True)
merged_counts['Flagged_situations'] = merged_counts['overstock_counts'] + merged_counts['understock_count'] + merged_counts['out_of_stock_count']
print(merged_counts.to_string(index=False))

warehouse_id  overstock_counts  understock_count  out_of_stock_count  Flagged_situations
          W1               0.0                57                 2.0                59.0
          W2               0.0                58                 0.0                58.0
          W3               0.0                47                 3.0                50.0
          W4               0.0                49                 0.0                49.0
          W5               0.0                48                 1.0                49.0
          W6               0.0                51                 1.0                52.0
          W7               0.0                37                 0.0                37.0
          W8               0.0                50                 0.0                50.0
          W9               0.0                51                 0.0                51.0
         W10               0.0                71                 1.0                72.0
         W11         

The code below evaluates the correlation coefficients between flagged situations and three varaibles Overstock_count,
 Understock_count and out_of_stock_counts and then prints the correlation coefficient.

In [8]:
correlation_coeff_os=merged_counts['Flagged_situations'].corr(merged_counts['overstock_counts'])
correlation_coeff_us=merged_counts['Flagged_situations'].corr(merged_counts['understock_count'])
correlation_coeff_ofs=merged_counts['Flagged_situations'].corr(merged_counts['out_of_stock_count'])
print(correlation_coeff_os)
print(correlation_coeff_us)
print(correlation_coeff_ofs)

-0.2499006448477867
0.9967464684222672
0.3611331507224739


**Ranking warehouses**
The ranking of warehouses is done based on the correlation coefficients. To avoid conflicting ranks"warehouse capacity" is considered as the 4th parameter.

In [9]:
warehouse_ranking_data = pd.merge(warehouse_information, understock_situations_sorted, on='warehouse_id', how='left')
warehouse_ranking_data = pd.merge(warehouse_ranking_data, overstock_countsdf, on='warehouse_id', how='left')
warehouse_ranking_data = pd.merge(warehouse_ranking_data,out_of_stock_counts, on='warehouse_id', how='left')
warehouse_ranking_data.fillna(0, inplace=True)
def rank_warehouses(row):
    return (
        -row['understock_count'],
        -row['out_of_stock_count'],
        -row['overstock_counts'],
        row['capacity']
    )
warehouse_ranking_data['ranking_score'] = warehouse_ranking_data.apply(rank_warehouses, axis=1)
warehouse_ranking_data['rank'] = warehouse_ranking_data['ranking_score'].rank(ascending=False, method='first')
warehouse_ranking_data = warehouse_ranking_data.sort_values(by='rank')
print(warehouse_ranking_data[['rank','warehouse_id', 'ranking_score']].to_string(index=False))

 rank warehouse_id           ranking_score
  1.0          W12 (-37, -0.0, -0.0, 6209)
  2.0           W7 (-37, -0.0, -0.0, 5225)
  3.0          W18 (-42, -0.0, -1.0, 3322)
  4.0          W16 (-46, -0.0, -0.0, 3692)
  5.0           W3 (-47, -3.0, -0.0, 3775)
  6.0          W13 (-48, -1.0, -0.0, 6637)
  7.0           W5 (-48, -1.0, -0.0, 3352)
  8.0           W4 (-49, -0.0, -0.0, 5496)
  9.0           W8 (-50, -0.0, -0.0, 7984)
 10.0           W9 (-51, -0.0, -0.0, 4254)
 11.0           W6 (-51, -1.0, -0.0, 5636)
 12.0          W20 (-53, -0.0, -0.0, 3214)
 13.0          W14 (-54, -0.0, -0.0, 6244)
 14.0          W11 (-55, -1.0, -0.0, 3422)
 15.0           W1 (-57, -2.0, -0.0, 6669)
 16.0           W2 (-58, -0.0, -0.0, 6730)
 17.0          W19 (-61, -1.0, -0.0, 7260)
 18.0          W15 (-68, -1.0, -0.0, 4460)
 19.0          W17 (-69, -1.0, -0.0, 7694)
 20.0          W10 (-71, -1.0, -0.0, 6800)


**Data Preparation for Visualisations**

In [10]:
merged_data = pd.merge(stock_history_df,order_history_df, on=['date', 'warehouse_id', 'product_id'], how='outer')
merged_data['stock_level'].fillna(0, inplace=True)
merged_data['quantity_ordered'].fillna(0, inplace=True)
merged_data = merged_data[['date', 'warehouse_id', 'product_id', 'stock_level', 'quantity_ordered']]
merged_data.to_csv('output_data.csv', index=False)

In [17]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [19]:
from google.colab import output
output.enable_custom_widget_manager()

**Visualizations**

*  The code below facilitates data exploration and visualization.
* Plotly library is used to create interactive line charts to visualize stock levels and quantity ordered data for specific products and warehouses.

Additionally it incorporates dropdown menus(widgets) for selecting particular warehouse and product.Moreover it annotates the line charts with understock,overstock and out of stock conditions.

In [23]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
data = pd.read_csv('/content/output_data.csv')
annotation_data = pd.read_csv('/understock_situations.csv')
annotation_data_1 = pd.read_csv('/out_of_stock_situations.csv')
annotation_data_2 = pd.read_csv('/OUTPUT1 (1).csv')
unique_warehouse_ids = data['warehouse_id'].unique()
unique_product_ids = data['product_id'].unique()
warehouse_dropdown = widgets.Dropdown(options=unique_warehouse_ids, description='Warehouse ID:')
product_dropdown = widgets.Dropdown(options=unique_product_ids, description='Product ID:')
def update_graph(warehouse_id, product_id):
    filtered_data = data[(data['warehouse_id'] == warehouse_id) & (data['product_id'] == product_id)]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data['date'], y=filtered_data['stock_level'], mode='markers', name='Stock Level'))
    fig.add_trace(go.Scatter(x=filtered_data['date'], y=filtered_data['quantity_ordered'], mode='markers', name='Quantity Ordered'))
    for index, row in annotation_data.iterrows():
        specific_date = row['date']
        specific_warehouse_id = row['warehouse_id']
        specific_product_id = row['product_id']
        if specific_warehouse_id == warehouse_id and specific_product_id == product_id:
            fig.add_annotation(x=specific_date, y=filtered_data[(filtered_data['date'] == specific_date)]['stock_level'].values[0],
                               text="Under_stock", showarrow=True, arrowhead=1)
    for index, row in annotation_data_1.iterrows():
        specific_date = row['date']
        specific_warehouse_id = row['warehouse_id']
        specific_product_id = row['product_id']
        if specific_warehouse_id == warehouse_id and specific_product_id == product_id:
            y_offset = 20
            x_offset = 20
            fig.add_annotation(x=specific_date, y=filtered_data[(filtered_data['date'] == specific_date)]['stock_level'].values[0]+ y_offset+x_offset,
                               text="Out_of_stock", showarrow=True, arrowhead=1)
    for index, row in annotation_data_2.iterrows():
        specific_date = row['date']
        specific_warehouse_id = row['warehouse_id']
        specific_product_id = row['product_id']
        if specific_warehouse_id == warehouse_id and specific_product_id == product_id:
            y_offset = 20
            x_offset = -20
            fig.add_annotation(x=specific_date, y=filtered_data[(filtered_data['date'] == specific_date)]['stock_level'].values[0]+ y_offset+x_offset,
                               text="Overstock", showarrow=True, arrowhead=1)
    fig.update_layout(title=f"Product {product_id} in Warehouse {warehouse_id}")
    display(go.FigureWidget(fig))
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_graph(warehouse_dropdown.value, product_dropdown.value)
display(warehouse_dropdown)
display(product_dropdown)
update_graph(warehouse_dropdown.value, product_dropdown.value)
warehouse_dropdown.observe(on_change)
product_dropdown.observe(on_change)


Dropdown(description='Warehouse ID:', options=('W1', 'W16', 'W3', 'W11', 'W13', 'W14', 'W15', 'W2', 'W20', 'W4…

Dropdown(description='Product ID:', options=('P10', 'P15', 'P6', 'P18', 'P13', 'P19', 'P9', 'P2', 'P16', 'P17'…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Stock Level',
              'type': 'sc…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Stock Level',
              'type': 'sc…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Stock Level',
              'type': 'sc…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Stock Level',
              'type': 'sc…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Stock Level',
              'type': 'sc…

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()